In [93]:
import pandas as pd
import requests
import os
import json
import math

In [123]:
df = pd.read_csv("./output/developerAddress.csv")
first_half = df[:4000]
other_half = df[4000:]
other_half

,DEVELOPER_ADDRESS
4000,"84 Merton Street, Toronto, ON M4S 1A1"
4001,"Zeballos 313, Rosario"
4002,"601 Poydras St. Ste 1500\nNew Orleans, LA 70130"
4003,"Washington D.C., USA"
4004,"4655 Great America Parkway, Suite 300\nSanta C..."
...,...
8586,Viker Games Limited\n200 Union Street\nLondon\...
8587,"New York City, NY"
8588,"Украина, г.Сумы"
8589,Ankara/Turkey


In [127]:
len(other_half)

4591

In [109]:
developer_addr = other_half.loc[11, "DEVELOPER_ADDRESS"]
developer_addr

'Andrey Nikanorov\nP.O. box 1\nSt. Petersburg\nRussia\n197720'

In [125]:
from decouple import config
API_KEY = config('API_KEY')
developer_addr = other_half.loc[4000, "DEVELOPER_ADDRESS"]
url = f"https://maps.googleapis.com/maps/api/place/autocomplete/json?input={developer_addr}&key={API_KEY}"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

In [126]:
data = json.loads(response.text)
data

{'predictions': [{'description': '84 Merton Street, Toronto, ON M4S 1A1, Canada',
   'matched_substrings': [{'length': 2, 'offset': 0},
    {'length': 13, 'offset': 3},
    {'length': 7, 'offset': 18},
    {'length': 2, 'offset': 27},
    {'length': 7, 'offset': 30}],
   'place_id': 'ChIJ31mxmkczK4gRGkX1WRynn0g',
   'reference': 'ChIJ31mxmkczK4gRGkX1WRynn0g',
   'structured_formatting': {'main_text': '84 Merton Street',
    'main_text_matched_substrings': [{'length': 2, 'offset': 0},
     {'length': 13, 'offset': 3}],
    'secondary_text': 'Toronto, ON M4S 1A1, Canada',
    'secondary_text_matched_substrings': [{'length': 7, 'offset': 0},
     {'length': 2, 'offset': 9},
     {'length': 7, 'offset': 12}]},
   'terms': [{'offset': 0, 'value': '84'},
    {'offset': 3, 'value': 'Merton Street'},
    {'offset': 18, 'value': 'Toronto'},
    {'offset': 27, 'value': 'ON'},
    {'offset': 30, 'value': 'M4S 1A1'},
    {'offset': 39, 'value': 'Canada'}],
   'types': ['premise', 'geocode']}],
 's

In [134]:
requests_fired = 0;
for i in range(5000, len(other_half) + 4000):
    developer_addr = other_half.loc[i, "DEVELOPER_ADDRESS"]
    if (pd.isnull(developer_addr)):
        print("NaN")
        continue
    formmated = developer_addr.replace("#", " ")
    url = f"https://maps.googleapis.com/maps/api/place/autocomplete/json?input={formmated}&key={API_KEY}"
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    data = json.loads(response.text)
    if not 'predictions' in data.keys():
        other_half.at[i, "DEVELOPER_ADDRESS_PRED"] = "NaN"
        continue
    if (data["predictions"] == []):
        other_half.at[i, "DEVELOPER_ADDRESS_PRED"] = "undefined"
        continue
    prediction = data["predictions"][0]["description"]
    requests_fired +=1
    other_half.at[i, "DEVELOPER_ADDRESS"] = formmated
    other_half.at[i, "DEVELOPER_ADDRESS_PRED"] = prediction
other_half.head(20)

,DEVELOPER_ADDRESS,DEVELOPER_ADDRESS_PRED
4000,"84 Merton Street, Toronto, ON M4S 1A1","84 Merton Street, Toronto, ON M4S 1A1, Canada"
4001,"Zeballos 313, Rosario","Zeballos 313, Rosario, Santa Fe Province, Arge..."
4002,"601 Poydras St. Ste 1500\nNew Orleans, LA 70130","601 Poydras St, New Orleans, LA 70130, USA"
4003,"Washington D.C., USA","Washington D.C., DC, USA"
4004,"4655 Great America Parkway, Suite 300\nSanta C...","4655 Great America Parkway, Santa Clara, CA 95..."
4005,"Bangkok, Thailand","Bangkok, Thailand"
4006,"150-8554\nEbisu Subaru Bldg. 1-20-8, Ebisu, Sh...","Tully's Coffee Ebisu SUBARU Bldg. Shop, 1 Chom..."
4007,"REEF Technology\n601 Brickell Key Drive, Suite...",undefined
4008,"Via Fondacci, 269\n55054 Massarosa (LU)\nItaly","Via dei Fondacci, 269, 55054 Montramito, Massa..."
4009,Kingdom of Saudi Arabia,Kingdom of Saudi Arabia


In [135]:
file_path = "./other_half.csv"
other_half.to_csv(path_or_buf=file_path, index=False);
requests_fired

854

In [132]:
other_half.loc[4000, "DEVELOPER_ADDRESS_PRED"]

'84 Merton Street, Toronto, ON M4S 1A1, Canada'

In [133]:
other_half.head(4000)

,DEVELOPER_ADDRESS,DEVELOPER_ADDRESS_PRED
4000,"84 Merton Street, Toronto, ON M4S 1A1","84 Merton Street, Toronto, ON M4S 1A1, Canada"
4001,"Zeballos 313, Rosario",NaN
4002,"601 Poydras St. Ste 1500\nNew Orleans, LA 70130",NaN
4003,"Washington D.C., USA",NaN
4004,"4655 Great America Parkway, Suite 300\nSanta C...",NaN
...,...,...
7995,"Imperia Online JSC\n69 Bulgaria Blvd, Manastir...",NaN
7996,17. novembra 8\n911 01 Trenčín\nSLOVAK REPUBLIC,NaN
7997,台灣台北市中山區南京東路三段168號9樓之5,NaN
7998,"K. 450, KV. 42 G. ZELENOGRAD, RUSSIA",NaN
